In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog
import folium, webbrowser, csv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import json
import os
import shutil
import datetime
import random
import re

#User and Data Storage
# Load users from file if exists
def load_users_from_file():
    users_data = []
    if os.path.exists('users.json'):
        try:
            file = open('users.json', 'r')
            data = json.load(file)
            file.close()
            
            # Handle both old dictionary format and new list format
            if isinstance(data, dict):
                # Convert old format to new format
                for username, user_info in data.items():
                    if isinstance(user_info, (list, tuple)) and len(user_info) >= 2:
                        users_data.append({
                            "username": username,
                            "password": user_info[0],
                            "id": "0000",  # Default ID for migrated users
                            "role": user_info[1]
                        })
            elif isinstance(data, list):
                users_data = data
                
        except Exception as e:
            print(f"Error loading users: {e}")
            users_data = []
            
    return users_data

def save_users_to_file():
    file = open('users.json', 'w')
    json.dump(users, file)
    file.close()

def load_minerals_from_file():
    minerals_data = {}
    if os.path.exists('minerals.json'):
        file = open('minerals.json', 'r')
        minerals_data = json.load(file)
        file.close()
        # Convert coordinate lists back to proper format
        for mineral_name in minerals_data:
            mineral_info = minerals_data[mineral_name]
            coordinates = mineral_info[3]
            minerals_data[mineral_name] = (mineral_info[0], mineral_info[1], mineral_info[2], coordinates)
    return minerals_data

def save_minerals_to_file():
    file = open('minerals.json', 'w')
    json.dump(minerals, file)
    file.close()

def load_countries_from_file():
    countries_data = {}
    if os.path.exists('countries.json'):
        file = open('countries.json', 'r')
        countries_data = json.load(file)
        file.close()
    return countries_data

def save_countries_to_file():
    file = open('countries.json', 'w')
    json.dump(country_profiles, file)
    file.close()

def load_favorites_from_file():
    favorites_data = []
    if os.path.exists('favorites.json'):
        file = open('favorites.json', 'r')
        favorites_data = json.load(file)
        file.close()
    return favorites_data

def save_favorites_to_file():
    file = open('favorites.json', 'w')
    json.dump(favorites, file)
    file.close()

# Load data when program starts
users = load_users_from_file()
if not users:  # If no users file exists, use default data
    users = [
        {"username": "Ina10", "password": "password", "id": "0405", "role": "Administrator"}
    ]

minerals = load_minerals_from_file()
if not minerals:  # If no minerals file exists, use default data
    minerals = {
        "Cobalt": ("Co", "Battery production", "Democratic Republic of Congo", [-4.0, 15.0]),
        "Lithium": ("Li", "Electric vehicle batteries", "Zimbabwe", [-20.0, 30.0]),
        "Manganese": ("Mn", "Steel production", "South Africa", [-28.0, 24.0]),
        "Graphite": ("C", "Battery anodes", "Mozambique", [-18.0, 35.0])
    }

country_profiles = load_countries_from_file()
if not country_profiles:  # If no countries file exists, use default data
    country_profiles = {
        "South Africa": (["Manganese", "Platinum", "Chromium"], 200, 7.5, "Kalahari Manganese Field"),
        "Zimbabwe": (["Lithium", "Gold", "Platinum"], 50, 4.0, "Bikita Lithium Project"),
        "Democratic Republic Of Congo": (["Cobalt", "Copper"], 300, 10.2, "Katanga Mining Operations"),
        "Mozambique": (["Graphite", "Coal"], 80, 3.1, "Balama Graphite Project")
    }

favorites = load_favorites_from_file()

ROLE_OPTIONS = ["Administrator", "Investor", "Researcher"]

#Username Validation by Role
def validate_username(username, role):
    """
    Validate username based on specific requirements:
    - Administrator: Must be "Ina10" (only for the default admin)
    - Investor/Researcher: Can be any username
    """
    username = username.strip()
    
    # Only enforce username format for Administrator role
    if role == "Administrator":
        if username != "Ina10":
            return False, "Administrator username must be 'Ina10'"
    
    # For Investor and Researcher roles, any username is allowed
    return True, "Valid username"

def detect_role_from_username(username):
    """
    Automatically detect role from username
    - "Ina10" is always Administrator
    - For other usernames, check the stored role
    """
    username = username.strip()
    
    # "Ina10" is always Administrator
    if username == "Ina10":
        return "Administrator"
    
    # For other usernames, find their role in the users list
    for user in users:
        if user["username"] == username:
            return user["role"]
    
    return None

#Market Price Data for Candlestick Chart
def generate_market_data():
    """Generate sample market price data for minerals"""
    minerals_list = ["Cobalt", "Lithium", "Manganese", "Graphite"]
    market_data = {}
    
    for mineral in minerals_list:
        prices = []
        current_date = datetime.datetime(2024, 1, 1)
        base_price = random.uniform(50, 200)  # Base price between $50-$200
        
        for week in range(52):  # 52 weeks of data
            open_price = base_price + random.uniform(-10, 10)
            close_price = open_price + random.uniform(-15, 15)
            high_price = max(open_price, close_price) + random.uniform(5, 20)
            low_price = min(open_price, close_price) - random.uniform(5, 20)
            
            # Ensure prices are positive
            open_price = max(open_price, 10)
            close_price = max(close_price, 10)
            high_price = max(high_price, 10)
            low_price = max(low_price, 10)
            
            prices.append({
                'date': current_date,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'volume': random.randint(1000, 10000)
            })
            
            current_date += datetime.timedelta(days=7)
            # Update base price for next week with some trend
            base_price = close_price + random.uniform(-5, 5)
        
        market_data[mineral] = prices
    
    return market_data

# Generate market data
market_prices = generate_market_data()

#NEW FEATURES

# 1. Search Function
def search_minerals():
    search_term = search_entry.get().strip().lower()
    if not search_term:
        populate_minerals()
        return
    
    for widget in mineral_content.winfo_children():
        widget.destroy()
    
    
    title_frame = tk.Frame(mineral_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text=f"🔍 Search Results for: '{search_term}'", font=("Segoe UI", 28, "bold"),
             fg="#FFD700", bg="#20243b").pack()
    
    # Content container with centered alignment
    content_frame = tk.Frame(mineral_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)  # Increased padding for centering
    
    found_any = False
    for name, details in minerals.items():
        if (search_term in name.lower() or 
            search_term in details[0].lower() or 
            search_term in details[1].lower() or 
            search_term in details[2].lower()):
            
            found_any = True
        
            card_container = tk.Frame(content_frame, bg="#20243b")
            card_container.pack(fill="x", pady=10)
            
            card = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
            card.pack(fill="x", pady=5, padx=150)  # Add horizontal padding to center the cards
            
            # Favorite star
            star = "★" if name in favorites else "☆"
            title_frame_card = tk.Frame(card, bg="#2b2f4b")
            title_frame_card.pack(fill="x")
            
            tk.Label(title_frame_card, text=name, font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#2b2f4b").pack(side="left")
            fav_btn = tk.Button(title_frame_card, text=star, font=("Segoe UI", 16), fg="#FFD700", bg="#2b2f4b", 
                              bd=0, command=lambda n=name: toggle_favorite(n))
            fav_btn.pack(side="right")
            
            info_frame = tk.Frame(card, bg="#2b2f4b")
            info_frame.pack(fill="x", padx=10)
            
            tk.Label(info_frame, text=f"🔤 Symbol: {details[0]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)
            tk.Label(info_frame, text=f"🏭 Main Use: {details[1]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)
            tk.Label(info_frame, text=f"🌍 Country: {details[2]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)
    
    if not found_any:
        tk.Label(content_frame, text="No minerals found matching your search.", 
                fg="white", bg="#20243b", font=("Segoe UI", 16)).pack(pady=50)

# 2. Favorites System
def toggle_favorite(mineral_name):
    if mineral_name in favorites:
        favorites.remove(mineral_name)
        show_notification(f"Removed {mineral_name} from favorites")
    else:
        favorites.append(mineral_name)
        show_notification(f"Added {mineral_name} to favorites")
    save_favorites_to_file()
    if hasattr(search_entry, 'get') and search_entry.get().strip():
        search_minerals()
    else:
        populate_minerals()

def show_favorites():
    for widget in mineral_content.winfo_children():
        widget.destroy()
    
    title_frame = tk.Frame(mineral_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="⭐ Favorite Minerals", font=("Segoe UI", 28, "bold"),
             fg="#FFD700", bg="#20243b").pack()
    
    # Content container with centered alignment
    content_frame = tk.Frame(mineral_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)  # Increased padding for centering
    
    if not favorites:
        tk.Label(content_frame, text="No favorites yet. Click the star icon on any mineral to add it!", 
                fg="white", bg="#20243b", font=("Segoe UI", 16)).pack(pady=50)
        return
    
    for name in favorites:
        if name in minerals:
            details = minerals[name]
            # Center each card by making it smaller and centering
            card_container = tk.Frame(content_frame, bg="#20243b")
            card_container.pack(fill="x", pady=10)
            
            card = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
            card.pack(fill="x", pady=5, padx=150)  # Add horizontal padding to center the cards
            
            title_frame_card = tk.Frame(card, bg="#2b2f4b")
            title_frame_card.pack(fill="x")
            
            tk.Label(title_frame_card, text=name, font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#2b2f4b").pack(side="left")
            fav_btn = tk.Button(title_frame_card, text="★", font=("Segoe UI", 16), fg="#FFD700", bg="#2b2f4b", 
                              bd=0, command=lambda n=name: toggle_favorite(n))
            fav_btn.pack(side="right")
            
            info_frame = tk.Frame(card, bg="#2b2f4b")
            info_frame.pack(fill="x", padx=10)
            
            tk.Label(info_frame, text=f"🔤 Symbol: {details[0]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)
            tk.Label(info_frame, text=f"🏭 Main Use: {details[1]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)
            tk.Label(info_frame, text=f"🌍 Country: {details[2]}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14)).pack(anchor="w", pady=5)

# 3. Quick Statistics
def show_stats():
    total_countries = len(country_profiles)
    total_minerals = len(minerals)
    total_favorites = len(favorites)
    
    # Find top producer
    top_producer = max(country_profiles.items(), key=lambda x: x[1][1]) if country_profiles else ("None", ([""], 0, 0, ""))
    
    stats_window = tk.Toplevel(root)
    stats_window.title("📊 Quick Statistics")
    stats_window.geometry("400x300")
    stats_window.config(bg="#20243b")
    stats_window.transient(root)
    stats_window.grab_set()
    
    # Center content in stats window
    center_frame = tk.Frame(stats_window, bg="#20243b")
    center_frame.place(relx=0.5, rely=0.5, anchor='center')
    
    tk.Label(center_frame, text="📊 Quick Statistics", font=("Segoe UI", 24, "bold"), 
             fg="#FFD700", bg="#20243b").pack(pady=20)
    
    stats_frame = tk.Frame(center_frame, bg="#20243b")
    stats_frame.pack(fill="both", expand=True, padx=30, pady=10)
    
    stats_data = [
        f"🌍 Total Countries: {total_countries}",
        f"💎 Total Minerals: {total_minerals}",
        f"⭐ Favorite Minerals: {total_favorites}",
        f"🏆 Top Producer: {top_producer[0]} ({top_producer[1][1]} Mt)"
    ]
    
    for stat in stats_data:
        tk.Label(stats_frame, text=stat, fg="white", bg="#20243b", 
                font=("Segoe UI", 14), justify="left").pack(anchor="w", pady=8)
    
    tk.Button(center_frame, text="Close", command=stats_window.destroy,
              bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), 
              padx=20, pady=8).pack(pady=20)

# 4. Notification System
def show_notification(message):
    notification = tk.Toplevel(root)
    notification.geometry("300x80")
    notification.title("Notification")
    notification.config(bg="#2b2f4b")
    notification.transient(root)
    notification.overrideredirect(True)  # Remove window decorations
    
    # Position near center
    x = root.winfo_x() + (root.winfo_width() // 2) - 150
    y = root.winfo_y() + 100
    notification.geometry(f"+{x}+{y}")
    
    tk.Label(notification, text=message, fg="white", bg="#2b2f4b", 
             font=("Segoe UI", 11), wraplength=280).pack(expand=True, fill="both", padx=10, pady=10)
    
    notification.after(2000, notification.destroy)  # Auto-close after 2 seconds

# 5. Data Backup
def backup_data():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_folder = f"backup_{timestamp}"
    
    if not os.path.exists(backup_folder):
        os.makedirs(backup_folder)
    
    files_to_backup = ['users.json', 'minerals.json', 'countries.json', 'favorites.json']
    backed_up = 0
    
    for file in files_to_backup:
        if os.path.exists(file):
            shutil.copy(file, os.path.join(backup_folder, file))
            backed_up += 1
    
    show_notification(f"Backup created: {backup_folder} ({backed_up} files)")
    return backup_folder

# 6. Data Validation
def validate_number(input_str, field_name):
    try:
        float(input_str)
        return True
    except ValueError:
        messagebox.showerror("Error", f"Please enter a valid number for {field_name}")
        return False

# 7. Enhanced Export with Progress
def export_with_progress():
    progress_window = tk.Toplevel(root)
    progress_window.title("Exporting...")
    progress_window.geometry("300x100")
    progress_window.config(bg="#20243b")
    progress_window.transient(root)
    progress_window.grab_set()
    
    # Center content in progress window
    center_frame = tk.Frame(progress_window, bg="#20243b")
    center_frame.place(relx=0.5, rely=0.5, anchor='center')
    
    tk.Label(center_frame, text="📤 Exporting data...", 
             fg="white", bg="#20243b", font=("Segoe UI", 12)).pack(pady=20)
    
    progress_bar = tk.Frame(center_frame, bg="#2b2f4b", height=10)
    progress_bar.pack(fill="x", padx=50, pady=10)
    progress_bar.progress = tk.Frame(progress_bar, bg="#FFD700", height=10, width=10)
    progress_bar.progress.pack(side="left")
    
    def animate_export():
        # Animate progress bar
        width = 10
        while width < 200:
            width += 10
            progress_bar.progress.config(width=width)
            progress_window.update()
            progress_window.after(50)
        
        # Do actual export
        export_minerals_csv()
        progress_window.destroy()
        show_notification("Export completed successfully!")
    
    progress_window.after(100, animate_export)

#Scrollable Frame Helper
class ScrollableFrame(tk.Frame):
    def __init__(self, container, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        self.canvas = tk.Canvas(self, bg="#20243b", highlightthickness=0)
        scrollbar = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas, bg="#20243b")

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=scrollbar.set)

        self.canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # Mousewheel support (Windows/macOS/Linux)
        self.canvas.bind_all("<MouseWheel>", self._on_mousewheel)   # Windows/Mac
        self.canvas.bind_all("<Button-4>", self._on_mousewheel)     # Linux up
        self.canvas.bind_all("<Button-5>", self._on_mousewheel)     # Linux down

    def _on_mousewheel(self, event):
        if getattr(event, "num", None) == 4:
            delta = -1
        elif getattr(event, "num", None) == 5:
            delta = 1
        else:
            # Normalize delta for Windows/macOS
            delta = -1 * int(event.delta / 120)
        self.canvas.yview_scroll(delta, "units")

#Navigation & Auth Functions
def show_frame(frame):
    for f in (login_frame, dashboard_frame, mineral_frame, country_frame, chart_frame, user_frame, about_frame):
        f.pack_forget()
    frame.pack(fill="both", expand=True)

def login_user():
    username = username_entry.get().strip()
    password = password_entry.get().strip()
    
    if not username or not password:
        messagebox.showerror("Error", "Username and password are required.")
        return
    
    # Automatically detect role from username
    detected_role = detect_role_from_username(username)
    if detected_role is None:
        messagebox.showerror("Error", "Invalid username or password.")
        return
    
    # Find user in the list
    user_found = None
    for user in users:
        if user["username"] == username and user["password"] == password:
            user_found = user
            break
    
    if user_found:
        # Verify the detected role matches the stored role
        stored_role = user_found["role"]
        if stored_role == detected_role:
            show_frame(dashboard_frame)
            welcome_label.config(text=f"Welcome {username} ({stored_role})")
            if stored_role == "Administrator":
                admin_btns_frame.pack(pady=8)
                backup_btn.pack(side="right", padx=8)
            else:
                admin_btns_frame.pack_forget()
                backup_btn.pack_forget()
            show_notification(f"Login successful! Welcome back, {username}!")
        else:
            messagebox.showerror("Error", f"Role mismatch! Your username suggests {detected_role}, but account is registered as {stored_role}.")
    else:
        messagebox.showerror("Error", "Invalid username or password.")

def logout_user():
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)
    admin_btns_frame.pack_forget()
    backup_btn.pack_forget()
    show_frame(login_frame)
    show_notification("Logged out successfully!")

#Minerals Page Functions
def populate_minerals():
    for widget in mineral_content.winfo_children():
        widget.destroy()
    
    title_frame = tk.Frame(mineral_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="💎 Minerals Database", font=("Segoe UI", 28, "bold"),
             fg="#FFD700", bg="#20243b").pack()
    
    # Search bar - centered with padding
    search_frame = tk.Frame(mineral_content, bg="#20243b")
    search_frame.pack(fill="x", pady=10)
    
    global search_entry
    search_entry = tk.Entry(search_frame, font=("Segoe UI", 12), width=30)
    search_entry.pack(side="left", padx=5)
    search_entry.bind('<Return>', lambda e: search_minerals())
    
    tk.Button(search_frame, text="🔍 Search", command=search_minerals, 
              bg="#FFD700", fg="#20243b", font=("Segoe UI", 10, "bold")).pack(side="left", padx=5)
    tk.Button(search_frame, text="⭐ Favorites", command=show_favorites,
              bg="#FFD700", fg="#20243b", font=("Segoe UI", 10, "bold")).pack(side="left", padx=5)
    tk.Button(search_frame, text="📊 Stats", command=show_stats,
              bg="#2C73D2", fg="white", font=("Segoe UI", 10, "bold")).pack(side="left", padx=5)
    
    # Content container with centered alignment
    content_frame = tk.Frame(mineral_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)  # Increased padding for centering
    
    for name, details in minerals.items():
        # Center each card by making it smaller and centering
        card_container = tk.Frame(content_frame, bg="#20243b")
        card_container.pack(fill="x", pady=10)
        
        card = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
        card.pack(fill="x", pady=5, padx=150)  # Add horizontal padding to center the cards
        
        # Title with favorite star
        title_frame_card = tk.Frame(card, bg="#2b2f4b")
        title_frame_card.pack(fill="x")
        
        tk.Label(title_frame_card, text=name, font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#2b2f4b").pack(side="left")
        
        star = "★" if name in favorites else "☆"
        fav_btn = tk.Button(title_frame_card, text=star, font=("Segoe UI", 16), fg="#FFD700", bg="#2b2f4b", 
                          bd=0, command=lambda n=name: toggle_favorite(n))
        fav_btn.pack(side="right")
        
        info_frame = tk.Frame(card, bg="#2b2f4b")
        info_frame.pack(fill="x", padx=10)
        
        tk.Label(info_frame, text=f"🔤 Symbol: {details[0]}", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)
        tk.Label(info_frame, text=f"🏭 Main Use: {details[1]}", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)
        tk.Label(info_frame, text=f"🌍 Country: {details[2]}", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)

def edit_minerals():
    for widget in mineral_content.winfo_children():
        widget.destroy()
    
    title_frame = tk.Frame(mineral_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="✏️ Edit Minerals", font=("Segoe UI", 28, "bold"), fg="#FFD700", bg="#20243b").pack()
    
    # Content container with centered alignment
    content_frame = tk.Frame(mineral_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)  # Increased padding for centering

    for name in list(minerals.keys()):
        details = minerals[name]
        # Center each card by making it smaller and centering
        card_container = tk.Frame(content_frame, bg="#20243b")
        card_container.pack(fill="x", pady=10)
        
        frame = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
        frame.pack(fill="x", pady=5, padx=150)  # Add horizontal padding to center the cards

        tk.Label(frame, text=name, font=("Segoe UI", 18, "bold"), fg="#FFD700", bg="#2b2f4b").pack(anchor="w", pady=10)

        input_frame = tk.Frame(frame, bg="#2b2f4b")
        input_frame.pack(fill="x", padx=10)

        tk.Label(input_frame, text="🔤 Symbol:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=0, column=0, sticky="w", pady=8)
        symbol_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=30)
        symbol_entry.insert(0, details[0])
        symbol_entry.grid(row=0, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="🏭 Main Use:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=1, column=0, sticky="w", pady=8)
        use_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=30)
        use_entry.insert(0, details[1])
        use_entry.grid(row=1, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="🌍 Country:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=2, column=0, sticky="w", pady=8)
        country_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=30)
        country_entry.insert(0, details[2])
        country_entry.grid(row=2, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="📍 Latitude:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=3, column=0, sticky="w", pady=8)
        lat_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=30)
        lat_entry.insert(0, details[3][0])
        lat_entry.grid(row=3, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="📍 Longitude:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=4, column=0, sticky="w", pady=8)
        lon_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=30)
        lon_entry.insert(0, details[3][1])
        lon_entry.grid(row=4, column=1, padx=10, pady=8, sticky="w")

        def save_changes(n=name, se=symbol_entry, ue=use_entry, ce=country_entry, la=lat_entry, lo=lon_entry):
            if not validate_number(la.get(), "latitude") or not validate_number(lo.get(), "longitude"):
                return
            minerals[n] = (se.get(), ue.get(), ce.get(), [float(la.get()), float(lo.get())])
            save_minerals_to_file()
            show_notification(f"{n} updated successfully!")
            populate_minerals()

        btn_frame = tk.Frame(frame, bg="#2b2f4b")
        btn_frame.pack(fill="x", pady=10)
        tk.Button(btn_frame, text="💾 Save Changes", command=save_changes, bg="#2C73D2", fg="white", 
                 font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(side="right", padx=10)

    btn_frame = tk.Frame(content_frame, bg="#20243b")
    btn_frame.pack(pady=20)
    tk.Button(btn_frame, text="⬅️ Back to View", command=populate_minerals, bg="red", fg="white", 
             font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack()

#Country Page Functions
def populate_countries():
    for widget in country_content.winfo_children():
        widget.destroy()
    
    title_frame = tk.Frame(country_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="🌍 Country Profiles", font=("Segoe UI", 28, "bold"),
             fg="#FFD700", bg="#20243b").pack()
    
    content_frame = tk.Frame(country_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20) 
    
    for country, details in country_profiles.items():
        card_container = tk.Frame(content_frame, bg="#20243b")
        card_container.pack(fill="x", pady=10)
        
        card = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
        card.pack(fill="x", pady=5, padx=150)
        
        tk.Label(card, text=country, font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#2b2f4b").pack(anchor="w", pady=8)
        
        info_frame = tk.Frame(card, bg="#2b2f4b")
        info_frame.pack(fill="x", padx=10)
        
        tk.Label(info_frame, text=f"💎 Minerals: " + ", ".join(details[0]), fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)
        tk.Label(info_frame, text=f"📊 Production: {details[1]} Mt", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)
        tk.Label(info_frame, text=f"💰 GDP: {details[2]}%", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)
        tk.Label(info_frame, text=f"🏗️ Project: {details[3]}", fg="white", bg="#2b2f4b", 
                font=("Segoe UI", 14)).pack(anchor="w", pady=5)

def edit_countries():
    for widget in country_content.winfo_children():
        widget.destroy()
    
    
    title_frame = tk.Frame(country_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="✏️ Edit Country Profiles", font=("Segoe UI", 28, "bold"), fg="#FFD700", bg="#20243b").pack()
    

    content_frame = tk.Frame(country_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)
    for country in list(country_profiles.keys()):
        details = country_profiles[country]
        # Center each card by making it smaller and centering
        card_container = tk.Frame(content_frame, bg="#20243b")
        card_container.pack(fill="x", pady=10)
        
        frame = tk.Frame(card_container, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
        frame.pack(fill="x", pady=5, padx=150) 

        tk.Label(frame, text=country, font=("Segoe UI", 18, "bold"), fg="#FFD700", bg="#2b2f4b").pack(anchor="w", pady=10)

        input_frame = tk.Frame(frame, bg="#2b2f4b")
        input_frame.pack(fill="x", padx=10)

        tk.Label(input_frame, text="💎 Minerals (comma separated):", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=0, column=0, sticky="w", pady=8)
        minerals_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=40)
        minerals_entry.insert(0, ", ".join(details[0]))
        minerals_entry.grid(row=0, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="📊 Production (Mt):", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=1, column=0, sticky="w", pady=8)
        production_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=40)
        production_entry.insert(0, details[1])
        production_entry.grid(row=1, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="💰 GDP (%):", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=2, column=0, sticky="w", pady=8)
        gdp_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=40)
        gdp_entry.insert(0, details[2])
        gdp_entry.grid(row=2, column=1, padx=10, pady=8, sticky="w")

        tk.Label(input_frame, text="🏗️ Key Project:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=3, column=0, sticky="w", pady=8)
        project_entry = tk.Entry(input_frame, font=("Segoe UI", 12), width=40)
        project_entry.insert(0, details[3])
        project_entry.grid(row=3, column=1, padx=10, pady=8, sticky="w")

        def save_country(cn=country, me=minerals_entry, pe=production_entry, ge=gdp_entry, pr=project_entry):
            if not validate_number(pe.get(), "production") or not validate_number(ge.get(), "GDP"):
                return
            country_profiles[cn] = ([m.strip() for m in me.get().split(",")], int(pe.get()), float(ge.get()), pr.get())
            save_countries_to_file()
            show_notification(f"{cn} updated successfully!")
            populate_countries()

        btn_frame = tk.Frame(frame, bg="#2b2f4b")
        btn_frame.pack(fill="x", pady=10)
        tk.Button(btn_frame, text="💾 Save Changes", command=save_country, bg="#2C73D2", fg="white", 
                 font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(side="right", padx=10)

    btn_frame = tk.Frame(content_frame, bg="#20243b")
    btn_frame.pack(pady=20)
    tk.Button(btn_frame, text="⬅️ Back to View", command=populate_countries, bg="red", fg="white", 
             font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack()

#CSV Export (Save As dialog)
def export_minerals_csv():
    filepath = filedialog.asksaveasfilename(
        defaultextension=".csv",
        filetypes=[("CSV files", "*.csv")],
        initialfile="minerals.csv",
        title="Save Minerals CSV"
    )
    if not filepath:
        return
    with open(filepath, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Symbol", "Use", "Country", "Latitude", "Longitude"])
        for name, d in minerals.items():
            writer.writerow([name, d[0], d[1], d[2], d[3][0], d[3][1]])
    show_notification(f"Minerals exported to:\n{filepath}")

def export_countries_csv():
    filepath = filedialog.asksaveasfilename(
        defaultextension=".csv",
        filetypes=[("CSV files", "*.csv")],
        initialfile="countries.csv",
        title="Save Countries CSV"
    )
    if not filepath:
        return
    with open(filepath, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Country", "Minerals", "Production (Mt)", "GDP (%)", "Project"])
        for c, d in country_profiles.items():
            writer.writerow([c, ", ".join(d[0]), d[1], d[2], d[3]])
    show_notification(f"Countries exported to:\n{filepath}")

#User Management (Admin only)
def manage_users():
    for widget in user_content.winfo_children():
        widget.destroy()

    title_frame = tk.Frame(user_content, bg="#20243b")
    title_frame.pack(fill="x", pady=20)
    tk.Label(title_frame, text="👥 Manage Users", font=("Segoe UI", 28, "bold"),
             fg="#FFD700", bg="#20243b").pack()
    
    content_frame = tk.Frame(user_content, bg="#20243b")
    content_frame.pack(fill="both", expand=True, padx=100, pady=20)  # Increased padding for centering

    # Add New User form
    add_frame = tk.Frame(content_frame, bg="#2b2f4b", bd=3, relief="ridge", padx=20, pady=15)
    add_frame.pack(fill="x", pady=15)
    tk.Label(add_frame, text="➕ Add New User", font=("Segoe UI", 18, "bold"), fg="#FFD700", bg="#2b2f4b").pack(anchor="w", pady=10)

    form_frame = tk.Frame(add_frame, bg="#2b2f4b")
    form_frame.pack(fill="x", padx=10)

    tk.Label(form_frame, text="👤 Username:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=0, column=0, sticky="w", pady=8)
    new_user_entry = tk.Entry(form_frame, font=("Segoe UI", 12), width=25)
    new_user_entry.grid(row=0, column=1, padx=10, pady=8, sticky="w")
    
    tk.Label(form_frame, text="🔑 Password:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=1, column=0, sticky="w", pady=8)
    new_pass_entry = tk.Entry(form_frame, show="*", font=("Segoe UI", 12), width=25)
    new_pass_entry.grid(row=1, column=1, padx=10, pady=8, sticky="w")
    
    tk.Label(form_frame, text="🎭 Role:", fg="white", bg="#2b2f4b", font=("Segoe UI", 12)).grid(row=2, column=0, sticky="w", pady=8)
    new_role_var = tk.StringVar(value="Investor")  # Default to Investor
    role_menu = tk.OptionMenu(form_frame, new_role_var, "Investor", "Researcher")
    role_menu.config(font=("Segoe UI", 12), width=22)
    role_menu.grid(row=2, column=1, padx=10, pady=8, sticky="w")

    def add_user():
        uname = new_user_entry.get().strip()
        pwd = new_pass_entry.get().strip()
        role = new_role_var.get()
        if not uname or not pwd:
            messagebox.showerror("Error", "Username and password required.")
            return
        
        # Check if username already exists
        for user in users:
            if user["username"] == uname:
                messagebox.showerror("Error", f"User '{uname}' already exists.")
                return
        
        # Generate a simple ID (you can make this more sophisticated)
        user_id = str(len(users) + 1).zfill(4)
        
        new_user = {
            "username": uname,
            "password": pwd,
            "id": user_id,
            "role": role
        }
        
        users.append(new_user)
        save_users_to_file()
        show_notification(f"User '{uname}' added successfully as {role}!")
        new_user_entry.delete(0, tk.END)
        new_pass_entry.delete(0, tk.END)
        manage_users()
    
    btn_frame = tk.Frame(add_frame, bg="#2b2f4b")
    btn_frame.pack(fill="x", pady=10)
    tk.Button(btn_frame, text="➕ Add User", command=add_user, bg="#2C73D2", fg="white", 
             font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(side="right", padx=10)

    # Existing users list
    users_frame = tk.Frame(content_frame, bg="#20243b")
    users_frame.pack(fill="both", expand=True, pady=20)
    
    if users:
        tk.Label(users_frame, text="📋 Existing Users", font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#20243b").pack(anchor="w", pady=10)
        
        for user in list(users):
            uname = user["username"]
            role = user["role"]
            user_id = user["id"]
            
            # Center each user card
            user_container = tk.Frame(users_frame, bg="#20243b")
            user_container.pack(fill="x", pady=8)
            
            frame = tk.Frame(user_container, bg="#2b2f4b", bd=2, relief="ridge", padx=15, pady=12)
            frame.pack(fill="x", pady=5, padx=150)  # Add horizontal padding to center the cards

            user_info_frame = tk.Frame(frame, bg="#2b2f4b")
            user_info_frame.pack(fill="x", pady=5)

            tk.Label(user_info_frame, text=f"👤 {uname}", fg="white", bg="#2b2f4b", 
                    font=("Segoe UI", 14, "bold")).pack(side="left", padx=10)
            tk.Label(user_info_frame, text=f"🎭 {role}", fg="#FFD700", bg="#2b2f4b",
                    font=("Segoe UI", 12)).pack(side="left", padx=20)
            tk.Label(user_info_frame, text=f"🆔 ID: {user_id}", fg="white", bg="#2b2f4b",
                    font=("Segoe UI", 12)).pack(side="left", padx=20)

            controls_frame = tk.Frame(frame, bg="#2b2f4b")
            controls_frame.pack(fill="x", pady=5)

            # Don't allow changing the admin username or role
            if uname == "Ina10":
                tk.Label(controls_frame, text="🔒 System Administrator (Cannot be modified)", 
                        fg="#FFD700", bg="#2b2f4b", font=("Segoe UI", 10, "bold")).pack(side="left", padx=10)
            else:
                role_var_local = tk.StringVar(value=role)
                # Only allow changing between Investor and Researcher (not Administrator)
                role_dropdown = tk.OptionMenu(controls_frame, role_var_local, "Investor", "Researcher")
                role_dropdown.config(font=("Segoe UI", 10), width=12)
                role_dropdown.pack(side="left", padx=10)

                def save_user(u=uname, rv=role_var_local):
                    # Update user role
                    for user_obj in users:
                        if user_obj["username"] == u:
                            user_obj["role"] = rv.get()
                            break
                    save_users_to_file()
                    show_notification(f"{u}'s role updated successfully!")
                    manage_users()

                def delete_user(u=uname):
                    if messagebox.askyesno("Confirm", f"Delete user {u}?"):
                        # Remove user from list
                        global users
                        users = [user_obj for user_obj in users if user_obj["username"] != u]
                        save_users_to_file()
                        show_notification(f"User {u} deleted successfully!")
                        manage_users()

                tk.Button(controls_frame, text="💾 Save", command=save_user, bg="#2C73D2", fg="white",
                         font=("Segoe UI", 10, "bold")).pack(side="right", padx=5)
                tk.Button(controls_frame, text="🗑️ Delete", command=delete_user, bg="red", fg="white",
                         font=("Segoe UI", 10, "bold")).pack(side="right", padx=5)

#Charts (Line / Pie / Bar / Candlestick)
def show_line_chart():
    # Only clear the chart display area, not the entire chart frame
    for widget in chart_display_area.winfo_children():
        widget.destroy()
    
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100, facecolor='#20243b')
    ax.set_facecolor('#2b2f4b')
    
    years = [2020, 2021, 2022, 2023, 2024]
    cobalt = [100, 110, 120, 130, 140]
    lithium = [40, 42, 47, 50, 55]
    manganese = [220, 230, 235, 240, 250]
    
    ax.plot(years, cobalt, label="Cobalt", linewidth=3, marker='o')
    ax.plot(years, lithium, label="Lithium", linewidth=3, marker='s')
    ax.plot(years, manganese, label="Manganese", linewidth=3, marker='^')
    
    ax.set_xlabel("Year", fontsize=12, color='white')
    ax.set_ylabel("Production (Mt)", fontsize=12, color='white')
    ax.set_title("Mineral Production Trends (2020-2024)", fontsize=14, color='white', pad=20)
    ax.legend(facecolor='#2b2f4b', edgecolor='white', labelcolor='white')
    ax.tick_params(colors='white')
    ax.grid(True, alpha=0.3)
    
    for spine in ax.spines.values():
        spine.set_color('white')
    
    canvas = FigureCanvasTkAgg(fig, master=chart_display_area)
    canvas.draw()
    canvas.get_tk_widget().pack(fill="both", expand=True, padx=20, pady=10)

def show_pie_chart():
    # Only clear the chart display area, not the entire chart frame
    for widget in chart_display_area.winfo_children():
        widget.destroy()
    
    fig, ax = plt.subplots(figsize=(8, 8), dpi=100, facecolor='#20243b')
    ax.set_facecolor('#2b2f4b')
    
    countries = list(country_profiles.keys())
    gdp = [d[2] for d in country_profiles.values()]
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    wedges, texts, autotexts = ax.pie(gdp, labels=countries, autopct="%1.1f%%", colors=colors, startangle=90)
    
    ax.set_title("GDP Distribution by Country", fontsize=14, color='white', pad=20)
    
    for text in texts:
        text.set_color('white')
        text.set_fontsize(11)
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    canvas = FigureCanvasTkAgg(fig, master=chart_display_area)
    canvas.draw()
    canvas.get_tk_widget().pack(fill="both", expand=True, padx=20, pady=10)

def show_bar_chart():
    # Only clear the chart display area, not the entire chart frame
    for widget in chart_display_area.winfo_children():
        widget.destroy()
    
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100, facecolor='#20243b')
    ax.set_facecolor('#2b2f4b')
    
    countries = list(country_profiles.keys())
    production = [d[1] for d in country_profiles.values()]
    
    bars = ax.bar(countries, production, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax.set_title("Mineral Production by Country", fontsize=14, color='white', pad=20)
    ax.set_ylabel("Production (Mt)", fontsize=12, color='white')
    
    ax.tick_params(colors='white')
    plt.xticks(rotation=25, color='white')
    
    for spine in ax.spines.values():
        spine.set_color('white')
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 5,
                f'{height}', ha='center', va='bottom', color='white', fontweight='bold')
    
    canvas = FigureCanvasTkAgg(fig, master=chart_display_area)
    canvas.draw()
    canvas.get_tk_widget().pack(fill="both", expand=True, padx=20, pady=10)

def show_candlestick_chart():
    # Only clear the chart display area, not the entire chart frame
    for widget in chart_display_area.winfo_children():
        widget.destroy()
    
    # Create selection frame for mineral choice
    selection_frame = tk.Frame(chart_display_area, bg="#20243b")
    selection_frame.pack(fill="x", pady=10)
    
    tk.Label(selection_frame, text="Select Mineral:", fg="white", bg="#20243b", 
             font=("Segoe UI", 12)).pack(side="left", padx=10)
    
    mineral_var = tk.StringVar(value="Cobalt")
    mineral_menu = tk.OptionMenu(selection_frame, mineral_var, "Cobalt", "Lithium", "Manganese", "Graphite")
    mineral_menu.config(font=("Segoe UI", 10), bg="#FFD700", fg="#20243b")
    mineral_menu.pack(side="left", padx=10)
    
    def update_candlestick():
        selected_mineral = mineral_var.get()
        data = market_prices[selected_mineral]
        
        # Clear previous chart
        for widget in chart_display_area.winfo_children():
            if widget != selection_frame:
                widget.destroy()
        
        fig, ax = plt.subplots(figsize=(12, 7), dpi=100, facecolor='#20243b')
        ax.set_facecolor('#2b2f4b')
        
        # Extract data for plotting
        dates = [item['date'] for item in data]
        opens = [item['open'] for item in data]
        highs = [item['high'] for item in data]
        lows = [item['low'] for item in data]
        closes = [item['close'] for item in data]
        
        # Plot candlesticks
        for i, (date, open_val, high_val, low_val, close_val) in enumerate(zip(dates, opens, highs, lows, closes)):
            color = 'green' if close_val >= open_val else 'red'
            
            # Draw the high-low line
            ax.plot([i, i], [low_val, high_val], color='white', linewidth=1)
            
            # Draw the open-close rectangle
            height = abs(close_val - open_val)
            bottom = min(open_val, close_val)
            rect = Rectangle((i-0.3, bottom), 0.6, height, 
                           facecolor=color, edgecolor='white', alpha=0.7)
            ax.add_patch(rect)
        
        # Customize the chart
        ax.set_title(f"{selected_mineral} Market Prices - Weekly Candlestick Chart (2024)", 
                    fontsize=16, color='white', pad=20)
        ax.set_ylabel("Price ($/ton)", fontsize=12, color='white')
        ax.set_xlabel("Week", fontsize=12, color='white')
        
        # Format x-axis to show month names
        ax.set_xticks(range(0, len(dates), 4))
        ax.set_xticklabels([date.strftime('%b') for date in dates[::4]], color='white')
        
        ax.tick_params(colors='white')
        ax.grid(True, alpha=0.3)
        
        for spine in ax.spines.values():
            spine.set_color('white')
        
        # Add legend
        ax.plot([], [], color='green', label='Bullish (Close ≥ Open)', linewidth=8)
        ax.plot([], [], color='red', label='Bearish (Close < Open)', linewidth=8)
        ax.legend(facecolor='#2b2f4b', edgecolor='white', labelcolor='white', loc='upper left')
        
        # Add some statistics
        current_price = closes[-1]
        price_change = current_price - opens[0]
        percent_change = (price_change / opens[0]) * 100
        
        stats_text = f"Current: ${current_price:.2f}\nYTD Change: {percent_change:+.1f}%"
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='#2b2f4b', 
                edgecolor='white', alpha=0.8), color='white')
        
        canvas = FigureCanvasTkAgg(fig, master=chart_display_area)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True, padx=20, pady=10)
    
    # Add update button
    update_btn = tk.Button(selection_frame, text="Update Chart", command=update_candlestick,
                         bg="#FFD700", fg="#20243b", font=("Segoe UI", 10, "bold"))
    update_btn.pack(side="left", padx=10)
    
    # Show initial chart
    update_candlestick()

#Map
def show_map():
    africa_map = folium.Map(location=[0, 20], zoom_start=3)
    for name, details in minerals.items():
        folium.Marker(location=details[3], popup=f"{name} ({details[0]}) - {details[1]}, {details[2]}").add_to(africa_map)
    filepath = "african_minerals_map.html"
    africa_map.save(filepath)
    show_notification(f"Map saved as '{filepath}'. Opening in browser...")
    webbrowser.open_new_tab(filepath)

# --- GUI Setup ---
root = tk.Tk()
root.title("Epikaizo - African Critical Minerals App")
root.geometry("1200x750")
root.config(bg="#20243b")

# Setup keyboard shortcuts
def setup_shortcuts(event):
    if event.state == 4 and event.keysym == 'q':  # Ctrl+Q
        logout_user()
    elif event.keysym == 'F1':  # F1
        show_frame(about_frame)
    elif event.state == 4 and event.keysym == 'f':  # Ctrl+F
        if hasattr(search_entry, 'focus'):
            search_entry.focus()

root.bind('<Key>', setup_shortcuts)

# Login Frame
login_frame = tk.Frame(root, bg="#20243b")
tk.Label(login_frame, text="Epikaizo", font=("Segoe UI", 38, "bold"), fg="#FFD700", bg="#20243b").pack(pady=20)
tk.Label(login_frame, text="Overshadowing the Mining Industry", font=("Segoe UI", 15, "italic"), fg="#FFFBEA", bg="#20243b").pack(pady=10)

# Add simple login instructions
instructions_frame = tk.Frame(login_frame, bg="#20243b")
instructions_frame.pack(pady=10)

instructions_text = "Enter Login details"
tk.Label(instructions_frame, text=instructions_text, fg="#FFFBEA", bg="#20243b", 
         font=("Segoe UI", 10)).pack()

form_frame = tk.Frame(login_frame, bg="#20243b"); form_frame.pack(pady=12)
tk.Label(form_frame, text="Username:", fg="#FFFBEA", bg="#20243b", font=("Segoe UI", 13)).grid(row=0, column=0, padx=12, pady=6, sticky="w")
username_entry = tk.Entry(form_frame, font=("Segoe UI", 13), width=20); username_entry.grid(row=0, column=1, padx=12, pady=6)
tk.Label(form_frame, text="Password:", fg="#FFFBEA", bg="#20243b", font=("Segoe UI", 13)).grid(row=1, column=0, padx=12, pady=6, sticky="w")
password_entry = tk.Entry(form_frame, show="*", font=("Segoe UI", 13), width=20); password_entry.grid(row=1, column=1, padx=12, pady=6)

# Only Login button
tk.Button(form_frame, text="Login", command=login_user, width=15, bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold")).grid(row=3, column=0, columnspan=2, padx=17, pady=20)

login_frame.pack(fill="both", expand=True)

# Dashboard Frame
dashboard_frame = tk.Frame(root, bg="#20243b")
welcome_label = tk.Label(dashboard_frame, text="", font=("Segoe UI", 20, "bold"), fg="#FFD700", bg="#20243b")
welcome_label.pack(pady=15)

btns_frame = tk.Frame(dashboard_frame, bg="#20243b"); btns_frame.pack(fill="both", expand=True, pady=8)
btns_frame.grid_columnconfigure(0, weight=1)
btns_frame.grid_columnconfigure(1, weight=1)
btns_frame.grid_rowconfigure(0, weight=1)
btns_frame.grid_rowconfigure(1, weight=1)
btns_frame.grid_rowconfigure(2, weight=1)
btns_frame.grid_rowconfigure(3, weight=1)

tk.Button(btns_frame, text="📊 Mineral Database", command=lambda: [show_frame(mineral_frame), populate_minerals()], 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 16, "bold"), height=3).grid(row=0, column=0, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="🌍 Country Profiles", command=lambda: [show_frame(country_frame), populate_countries()], 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 16, "bold"), height=3).grid(row=0, column=1, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="🗺️ Generate Map", command=show_map, 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 16, "bold"), height=3).grid(row=1, column=0, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="📈 Charts", command=lambda: show_frame(chart_frame), 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 16, "bold"), height=3).grid(row=1, column=1, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="💾 Export Minerals CSV", command=export_with_progress, 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 14, "bold"), height=3).grid(row=2, column=0, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="💾 Export Countries CSV", command=export_countries_csv, 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 14, "bold"), height=3).grid(row=2, column=1, padx=15, pady=10, sticky="nsew")
tk.Button(btns_frame, text="ℹ️ About", command=lambda: show_frame(about_frame), 
          bg="#FFD700", fg="#20243b", font=("Segoe UI", 16, "bold"), height=3).grid(row=3, column=0, columnspan=2, padx=15, pady=10, sticky="nsew")

admin_btns_frame = tk.Frame(dashboard_frame, bg="#20243b")
tk.Button(admin_btns_frame, text="✏️ Edit Minerals", command=lambda: [show_frame(mineral_frame), edit_minerals()], 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), height=2, padx=15).pack(side="left", padx=8)
tk.Button(admin_btns_frame, text="✏️ Edit Countries", command=lambda: [show_frame(country_frame), edit_countries()], 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), height=2, padx=15).pack(side="left", padx=8)
tk.Button(admin_btns_frame, text="👥 Manage Users", command=lambda: [show_frame(user_frame), manage_users()], 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), height=2, padx=15).pack(side="left", padx=8)

# Backup button (admin only)
backup_btn = tk.Button(admin_btns_frame, text="💾 Backup Data", command=backup_data,
                      bg="#4CAF50", fg="white", font=("Segoe UI", 12, "bold"), height=2, padx=15)

tk.Button(dashboard_frame, text="🚪 Logout (Ctrl+Q)", command=logout_user, bg="red", fg="white", font=("Segoe UI", 12, "bold"), width=15).pack(pady=20)

# Minerals Page
mineral_frame = tk.Frame(root, bg="#20243b")
mineral_scroll = ScrollableFrame(mineral_frame); mineral_scroll.pack(fill="both", expand=True, pady=10)
mineral_content = mineral_scroll.scrollable_frame
tk.Button(mineral_frame, text="⬅️ Return to Dashboard", command=lambda: show_frame(dashboard_frame), 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(pady=10)

# Country Page
country_frame = tk.Frame(root, bg="#20243b")
country_scroll = ScrollableFrame(country_frame); country_scroll.pack(fill="both", expand=True, pady=10)
country_content = country_scroll.scrollable_frame
tk.Button(country_frame, text="⬅️ Return to Dashboard", command=lambda: show_frame(dashboard_frame), 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(pady=10)

# Charts Page - FIXED STRUCTURE
chart_frame = tk.Frame(root, bg="#20243b")

# Chart buttons at the top
chart_buttons = tk.Frame(chart_frame, bg="#20243b")
chart_buttons.pack(fill="x", pady=10)

tk.Button(chart_buttons, text="📈 Line Graph", command=show_line_chart, bg="#FFD700", fg="#20243b",
          font=("Segoe UI", 12, "bold"), padx=15, pady=8).pack(side="left", padx=6)
tk.Button(chart_buttons, text="🥧 Pie Chart", command=show_pie_chart, bg="#FFD700", fg="#20243b",
          font=("Segoe UI", 12, "bold"), padx=15, pady=8).pack(side="left", padx=6)
tk.Button(chart_buttons, text="📊 Bar Chart", command=show_bar_chart, bg="#FFD700", fg="#20243b",
          font=("Segoe UI", 12, "bold"), padx=15, pady=8).pack(side="left", padx=6)
tk.Button(chart_buttons, text="🕯️ Candlestick", command=show_candlestick_chart, bg="#FFD700", fg="#20243b",
          font=("Segoe UI", 12, "bold"), padx=15, pady=8).pack(side="left", padx=6)

# Chart display area in the middle (this is where charts will appear)
chart_display_area = tk.Frame(chart_frame, bg="#20243b")
chart_display_area.pack(fill="both", expand=True, pady=10)

# Return button at the bottom
tk.Button(chart_frame, text="⬅️ Return to Dashboard", command=lambda: show_frame(dashboard_frame), 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(pady=10)

# User Management Page
user_frame = tk.Frame(root, bg="#20243b")
user_scroll = ScrollableFrame(user_frame); user_scroll.pack(fill="both", expand=True, pady=10)
user_content = user_scroll.scrollable_frame
tk.Button(user_frame, text="⬅️ Return to Dashboard", command=lambda: show_frame(dashboard_frame), 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(pady=10)

# About Page
about_frame = tk.Frame(root, bg="#20243b")
about_scroll = ScrollableFrame(about_frame); about_scroll.pack(fill="both", expand=True, pady=10)
about_content = about_scroll.scrollable_frame

tk.Label(about_content, text="ℹ️ About Epikaizo", font=("Segoe UI", 28, "bold"), fg="#FFD700", bg="#20243b").pack(pady=20)

content_frame = tk.Frame(about_content, bg="#20243b")
content_frame.pack(fill="both", expand=True, padx=40, pady=20)

about_text = (
    "Epikaizo — Overshadowing the Mining Industry\n\n"
    "This application provides a comprehensive interface for exploring critical mineral data, "
    "country production profiles, interactive charts and maps for the African mining sector.\n\n"
    
    "🌟 Key Features:\n"
    "• 🔐 Role-based access control (Administrator, Investor, Researcher)\n"
    "• 💎 Comprehensive mineral database with detailed information\n"
    "• 🌍 Country profiles with production statistics and economic data\n"
    "• 📈 Interactive charts and data visualization\n"
    "• 🕯️ Candlestick charts for market price analysis\n"
    "• 🗺️ Geographical mapping of mineral deposits\n"
    "• ✏️ Inline editing capabilities for administrators\n"
    "• 💾 Data export functionality (CSV format)\n"
    "• 👥 User management system\n"
    "• 🔍 Smart search and favorites system\n"
    "• 📊 Quick statistics dashboard\n"
    "• 💾 Automatic data backup\n\n"
    
    "🎯 Purpose:\n"
    "Epikaizo addresses the critical need for centralized, dynamic platform to house essential data "
    "about Africa's critical minerals sector. The application supports strategic planning, investment "
    "analysis, and market intelligence for stakeholders across the mining industry value chain.\n\n"
    
    "💡 Pro Tips:\n"
    "• Use Ctrl+F to quickly search minerals\n"
    "• Click the star icon to favorite minerals\n"
    "• Use Ctrl+Q to quickly logout\n"
    "• Press F1 to view this about page\n"
    "• Check candlestick charts for market trend analysis\n\n"
    
    "Default Admin Login:\n"
    "• Username: Ina10\n"
    "• Password: password\n\n"
    
    "Use the Dashboard to navigate between sections. Administrators have additional privileges "
    "to manage users and edit mineral and country data directly within the application."
)

tk.Label(content_frame, text=about_text, font=("Segoe UI", 13), fg="white", bg="#20243b", 
         justify="left", wraplength=1000).pack(pady=10)

tk.Button(about_frame, text="⬅️ Return to Dashboard", command=lambda: show_frame(dashboard_frame), 
          bg="#2C73D2", fg="white", font=("Segoe UI", 12, "bold"), padx=20, pady=8).pack(pady=15)

# Start on login page
show_frame(login_frame)

# Show welcome notification on start
root.after(1000, lambda: show_notification("Welcome to Epikaizo!"))

root.mainloop()